# Data Preprocessing Tools

## Dataset

### Layout

* Columns: 4
	* Country
    * Age
    * Salary
    * Purchased
* Rows: 10 observations
	* Each row represents one purchase
		* Features:
			* Age
            * Salary
            * Purchased
        * Categories:
            * Country

### Background

* Company gathered data on their customer's purchases

### Goals

* Demonstrate data preprocessing techniques on the dataset

## Import Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Import Dataset

In [2]:
dataset = pd.read_csv('Data.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [3]:
print(X)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 nan]
 ['France' 35.0 58000.0]
 ['Spain' nan 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


In [4]:
print(y)

['No' 'Yes' 'No' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes']


## Taking Care of Missing Data

* `SimpleImputer` class is used for taking care of missing data
* `np.nan` (i.e. `NaN`) represents replacing any missing values in a dataset
* `strategy = 'mean'` instructs an object to replace missing values with the mean value
* `transform` method applies transformation by adding missing values with column mean value


In [5]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer.fit(X[:, 1:3])
X[:, 1:3] = imputer.transform(X[:, 1:3])

In [6]:
print(X)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 63777.77777777778]
 ['France' 35.0 58000.0]
 ['Spain' 38.77777777777778 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


## Identify Missing Numbers in Dataset

In [7]:
missing_data = dataset.isnull().sum()

In [22]:
print(missing_data)

Country      0
Age          1
Salary       1
Purchased    0
dtype: int64


## Encode Categorical Data

### Encode Independent Variable(s)

* `ColumnTransformer` and `OneHotEncoder` classes are used to transform non-numerical columns into binary vectors of multiple transformed columns
* `transformers` object parameters:
  * Kind of transformation (i.e. `encoding`)
  * Type of encoding (i.e. `OneHotEncoder`)
  * Indexes of columns to encode
* `remainder` parameters:
  * `passthrough` means to keep other feature columns the same values
* `fit_transform` method connects an object to a matrix of features and performs transformation in one method call
  * It returns a transformed matrix of features with column one-hot encoded
  * It does not return a matrix of features as numpy array
  * One needs to call numpy `array` method to transform output into numpy array

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [0])], remainder = 'passthrough')
X = np.array(ct.fit_transform(X))

In [9]:
print(X)

[[1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [0.0 1.0 0.0 30.0 54000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 35.0 58000.0]
 [0.0 0.0 1.0 38.77777777777778 52000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 37.0 67000.0]]


### Encode Dependent Variable

* For `LabelEncoder` class, there are no parameters since one is encoding a single vector
* `fit_transform` method connects an object to the dependent variable vector and performs transformation in one method call
* Returns transformed dependent variable vector with column label encoded (i.e. transforming `Yes` and `No` to binary vectors of `0` and `1`)
* No numpy array is needed since it is a single column

In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [13]:
print(y)

[0 1 0 0 1 1 0 1 0 1]


## Split Dataset into Training Set and Test Set

* `train_test_split` function is used to split a dataset into 4 sets for training and test by matrix of features/independent variables and dependent variable vectors
* Function parameters:
    * 1: `X` matrix of features/independent variables
    * 2: `y` dependent variable vector
    * 3: `test_size` indicates split size for testing (80% training, 20% test)
    * 4: `random_state` indicates a split factor to ensure values are evenly taken into each set

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [15]:
print(X_train)

[[0.0 0.0 1.0 38.77777777777778 52000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 35.0 58000.0]]


In [16]:
print(X_test)

[[0.0 1.0 0.0 30.0 54000.0]
 [1.0 0.0 0.0 37.0 67000.0]]


In [17]:
print(y_train)

[0 1 0 0 1 1 0 1]


In [18]:
print(y_test)

[0 1]


## Feature Scaling

* Feature scale all rows and columns, but not the encoded categorical columns
* Features to scale are indexes 3 and 4
* Index 4 is not specified so the code is scalable to take all columns after index 3
* `fit` method computes the mean and standard deviation of the values
* `transform` method applies the numerator and denominator of the feature scaling formula
* `fit_transform` method performs both operations

In [19]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[:, 3:] = sc.fit_transform(X_train[:, 3:])
X_test[:, 3:] = sc.transform(X_test[:, 3:])

In [20]:
print(X_train)

[[0.0 0.0 1.0 -0.19159184384578545 -1.0781259408412425]
 [0.0 1.0 0.0 -0.014117293757057777 -0.07013167641635372]
 [1.0 0.0 0.0 0.566708506533324 0.633562432710455]
 [0.0 0.0 1.0 -0.30453019390224867 -0.30786617274297867]
 [0.0 0.0 1.0 -1.9018011447007988 -1.420463615551582]
 [1.0 0.0 0.0 1.1475343068237058 1.232653363453549]
 [0.0 1.0 0.0 1.4379472069688968 1.5749910381638885]
 [1.0 0.0 0.0 -0.7401495441200351 -0.5646194287757332]]


In [21]:
print(X_test)

[[0.0 1.0 0.0 -1.4661817944830124 -0.9069571034860727]
 [1.0 0.0 0.0 -0.44973664397484414 0.2056403393225306]]
